In [6]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('./combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], 
                 usecols = [0,1])
#df1 = pd.read_csv('./datasets/combined_data_2.txt', header = None, names = ['Cust_Id', 'Rating'], 
#                 usecols = [0,1])
#df2 = pd.read_csv('./datasets/combined_data_3.txt', sep = ",")

In [16]:
df.head()

,Cust_Id,Rating,Movie_Id
696,712664,5.0,3
697,1331154,4.0,3
698,2632461,3.0,3
699,44937,5.0,3
700,656399,4.0,3


In [ ]:
df['Rating'] = df['Rating'].astype(float)
df.iloc[::5000,:]

df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

# remove those Movie ID rows
df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)

In [15]:
#trimming
f = ['count','mean']

df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index


df = df[~df['Movie_Id'].isin(drop_movie_list)]
df = df[~df['Cust_Id'].isin(drop_cust_list)]


print(df.head())

     Cust_Id  Rating  Movie_Id
696   712664     5.0         3
697  1331154     4.0         3
698  2632461     3.0         3
699    44937     5.0         3
700   656399     4.0         3


In [46]:
df.to_csv('./datasets/cleaned_nf_dataset.csv')

In [48]:
df.Cust_Id.max()

2649429